In [7]:
import pandas as pd
import numpy as np
import cv2
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import os

subjects = ["", "Kunal kashyap", "Elvis Presley"]


def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = gray.astype('uint8')
    gray = cv2.GaussianBlur(gray,(5,5),0)
    ret3,th3 = cv2.threshold(gray,0,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    # loding face cascade classifier model
    face_cascade = cv2.CascadeClassifier('F:/Python Data/opencv/course/opencv-face-recognition-python-master/face_recognition using LBP kunal/opencv-files/lbpcascade_frontalface.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5);

    if (len(faces) == 0):
        return None, None

    (x, y, w, h) = faces[0]
    return gray[y: y + w, x: x + h], faces[0]


def prepare_training_data(data_folder_path):
    dirs = os.listdir(data_folder_path)
    faces = []
    labels = []

    for dir_name in dirs:
        if not dir_name.startswith('s'):
            continue;
        label = int(dir_name.replace("s", ""))
        label = int(label)
        subject_dir_path = data_folder_path + "/" + dir_name

        subject_images_names = os.listdir(subject_dir_path)

        for image_name in subject_images_names:
            if image_name.startswith('.'):
                continue;
            image_path = subject_dir_path + '/' + image_name
            image = cv2.imread(image_path)
            cv2.imshow('training on image..', image)
            cv2.waitKey(100)
            # calling the  face detection model function \
            face, rect = detect_face(image)
            if face is not None:
                faces.append(face)
                labels.append(label)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    return faces, labels


print('preparing data....')
faces, labels = prepare_training_data("F:/Python Data/opencv/course/opencv-face-recognition-python-master/face_recognition using LBP kunal/training-data")
print('data prepared.')
print('total faces = ', len(faces))
print('total labels :', len(labels))

# Now we will use classification to classify which image is whom

# we can use any of these recognizer
# create our LBPH face recognizer
# face_recognizer = cv2.face_LBPHFaceRecognizer()

# or use EigenFaceRecognizer by replacing above line with
# face_recognizer = cv2.face_EigenFaceRecognizer()

# or use FisherFaceRecognizer by replacing above line with
# face_recognizer = cv2.face_FisherFaceRecognizer()

# but the best and fast is LBPH

import numpy as np

face_recognizer = cv2.face.LBPHFaceRecognizer_create()

face_recognizer.train(faces, labels=np.array(labels))


def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x + w, y + h), (0,0,255), 3)


# funtion to write name text on the image to identify who the person is
# passed x,y cordinates
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_ITALIC, 1.5, (0,0,255), 2)


# THIS FUNTION RECOGNIZE THE PERSON ON THE IMAGE
def predict(test_img):
    img = test_img.copy()
    face, rect = detect_face(img)

    label,confidence = face_recognizer.predict(face)
    print(type(label))
    print(type(confidence))
    label = np.array(label)
    confidence = np.array(confidence)
    label_text = subjects[label]
    draw_rectangle(img, rect)
    draw_text(img, label_text, rect[0], rect[1] - 5)
    return img


print('predicting images...')
test_img1 = cv2.imread('F:/Python Data/opencv/course/opencv-face-recognition-python-master/face_recognition using LBP kunal/test-data/test_data1.jpg')
test_img2 = cv2.imread('F:/Python Data/opencv/course/opencv-face-recognition-python-master/face_recognition using LBP kunal/test-data/test_data2.jpg')
# test_img1 = cv2.cvtColor(test_img1,cv2.COLOR_BGR2GRAY)
# test_img2 = cv2.cvtColor(test_img2,cv2.COLOR_BGR2GRAY)
predicted_img1 = predict(test_img1)
predicted_img2 = predict(test_img2)

print('prediction complete')
cv2.imshow(subjects[1], cv2.resize(predicted_img1, (400, 500)))
cv2.imshow(subjects[2], cv2.resize(predicted_img2, (400, 500)))
cv2.imwrite('box_image1.jpg', predicted_img1)
cv2.imwrite('box_image3.jpg', predicted_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()


preparing data....
data prepared.
total faces =  22
total labels : 22
predicting images...
<class 'int'>
<class 'float'>
<class 'int'>
<class 'float'>
prediction complete
